In [1]:
import sys

sys.path.append("../")

from analysis.scalar import HLFAnalysis
from models.nn import PaeBuilder

In [2]:
from utils import load_json, dump_json

from loaders.LHCO import LhcoRnDLoader

loader = LhcoRnDLoader.from_json("../configs/loader/default_rnd.json")
dataset_cfg = load_json("../configs/loader/default_dataset.json")
dataset = loader.make_full_dataset(**dataset_cfg)

In [3]:
import tensorflow.keras as tfk
import tensorflow as tf

from models.autoencoder import DenseAutoencoder
from models.flows import CondMAF, MAF

builder = PaeBuilder()

pae, ae_train, nf_train = builder.from_json('../configs/pae/default.json')

{'input_dim': 47, 'encoding_dim': 10, 'units': [30, 20, 15], 'hidden_activation': <function relu at 0x7fcf6a8bac10>, 'output_activation': <function sigmoid at 0x7fcf6a86c1f0>, 'weight_reg': <tensorflow.python.keras.regularizers.L1 object at 0x7fd010719fd0>, 'bias_reg': None, 'output_reg': None} {'n_dims': 10, 'n_layers': 5, 'units': [32, 32, 32, 32]}


In [4]:
task = HLFAnalysis(pae, dataset=dataset)


In [5]:
task.reweighting(estimator='gmm', fit_key='mjj_train')

In [6]:
task.train(ae_train,nf_train)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
Epoch 1/60
800/800 [==============================] - 5s 6ms/step - loss: 707.9423 - val_loss: 769.4918
Epoch 2/60
800/800 [==============================] - 1s 1ms/step - loss: 697.2883 - val_loss: 760.2806
Epoch 3/60
800/800 [==============================] - 1s 1ms/step - loss: 720.0026 - val_loss: 756.0623
Epoch 4/60
800/800 [==============================] - 1s 1ms/step - loss: 687.0767 - val_loss: 758.7836
Epoch 5/60
800/800 [==============================] - 1s 1ms/step - loss: 715.1197 - val_loss: 785.1694
Epoch 6/60
800/800 [==============================] - 1s 1ms/step - loss: 742.4886 - val_loss: 758.1429
Epoch 7/60
800/800 [==============================] - 1s 1ms/step - loss: 700.4642 - val_loss: 772.5308
Epoch 8/60
800/800 [==============================] - 1s 1ms/step - loss: 672.0136 - val_loss: 753.7998
Epoch 9/60
800/800 [==============================] - 1s 1ms/step - loss: 720.3678 - val_loss: 765.18

In [7]:
task.evaluate(prc = 90)

{'js_div_train': 0.043475602770994,
 'js_div_test': 0.07214356176889856,
 'sig_eff': 0.12357,
 'bkg_rej': 0.92357,
 'auc': 0.7139953695}

In [8]:
task.results

{'js_div_train': 0.043475602770994,
 'js_div_test': 0.07214356176889856,
 'sig_eff': 0.12357,
 'bkg_rej': 0.92357,
 'auc': 0.7139953695}

In [9]:
task.plot_training()

TypeError: 'NoneType' object is not subscriptable

In [11]:
task.plot_latent_space()

### From config dict

In [23]:
from models import OPTIMIZERS, ACTIVATIONS, REGULARIZERS, MODELS, CALLBACKS
pae_config = {
    'ae_model': 'dense_ae',
    'nf_model': 'maf',
    'AE:input_dim': 47,
    'AE:encoding_dim': 10,
    'AE:units': [30, 20 ,15],
    'AE:hidden_activation': 'relu',
    'AE:output_activation': 'sigmoid',
    'AE:weight_reg': {'l1': 1e-6},
    'AE:bias_reg': {},
    'AE:output_reg': {},
    'NF:n_dims': 10,
    'NF:n_layers':5, 
    'NF:units':[32 for i in range(4)],
    'ae_optimizer': 'adam',
    'nf_optimizer': 'adam',
    'ae_optimizer_kwargs': dict(lr=0.05),
    'nf_optimizer_kwargs': dict(lr=0.005),
    'ae_callbacks': ['reduce_lr_on_plateau'],
    'ae_callbacks_kwargs': [dict(factor=0.2, patience=10, verbose=1)],
    'nf_callbacks': ['reduce_lr_on_plateau'],
    'nf_callbacks_kwargs': [dict(factor=0.2, patience=5, verbose=1)],
    'ae_epochs': 60,
    'ae_batch_size': 200,
    'nf_epochs': 50,
    'nf_batch_size': 200
}

dump_json(pae_config, path='../configs/pae/default.json')



In [24]:
ae_config = {key.split(':')[1]:pae_config.pop(key) 
            for key in list(pae_config.keys()) 
            if 'AE:' in key}
nf_config = {key.split(':')[1]:pae_config.pop(key) 
             for key in list(pae_config.keys()) 
             if 'NF:' in key}
pae_config

{'ae_model': 'dense_ae',
 'nf_model': 'maf',
 'ae_optimizer': 'adam',
 'nf_optimizer': 'adam',
 'ae_optimizer_kwargs': {'lr': 0.05},
 'nf_optimizer_kwargs': {'lr': 0.005},
 'ae_callbacks': ['reduce_lr_on_plateau'],
 'ae_callbacks_kwargs': [{'factor': 0.2, 'patience': 10, 'verbose': 1}],
 'nf_callbacks': ['reduce_lr_on_plateau'],
 'nf_callbacks_kwargs': [{'factor': 0.2, 'patience': 5, 'verbose': 1}],
 'ae_epochs': 60,
 'ae_batch_size': 200,
 'nf_epochs': 50,
 'nf_batch_size': 200}

In [25]:
def interpret_args(config):
    for key in list(config):
        if 'model' in key:
            config[key] = MODELS[config[key]]
        elif 'optimizer' in key:
            if key+'_kwargs' in config:
                config[key] = OPTIMIZERS[config[key]](**config.pop(key+"_kwargs"))
            elif 'kwargs' not in key:
                config[key] = OPTIMIZERS[config[key]]()
        elif 'callbacks' in key and 'kwargs' not in key:
            if key+'_kwargs' in config:
                config[key] = [CALLBACKS[callback](**kwargs)
                                for callback, kwargs  
                                in zip(config[key],config.pop(key+"_kwargs"))
                                ]
            else:
                config[key] = [CALLBACKS[callback]()
                                for callback in config[key]
                                ] 
            
             

In [26]:
interpret_args(pae_config)

In [27]:
pae_config

{'ae_model': models.autoencoder.DenseAutoencoder,
 'nf_model': models.flows.MAF,
 'ae_optimizer': <tensorflow.python.keras.optimizer_v2.adam.Adam at 0x7fc7f3772460>,
 'nf_optimizer': <tensorflow.python.keras.optimizer_v2.adam.Adam at 0x7fc7f3772f40>,
 'ae_callbacks': [<tensorflow.python.keras.callbacks.ReduceLROnPlateau at 0x7fc7f3772220>],
 'nf_callbacks': [<tensorflow.python.keras.callbacks.ReduceLROnPlateau at 0x7fc7f37721c0>],
 'ae_epochs': 60,
 'ae_batch_size': 200,
 'nf_epochs': 50,
 'nf_batch_size': 200}

In [28]:
ae_type = pae_config.pop('ae_model')
ae_optim = pae_config.pop('ae_optimizer')
nf_type = pae_config.pop('nf_model')
nf_optim = pae_config.pop('nf_optimizer')
ae_train = {key.split('_')[1]:pae_config.pop(key) 
            for key in list(pae_config.keys()) 
            if 'ae' in key}
nf_train = {key.split('_')[1]:pae_config.pop(key) 
             for key in list(pae_config.keys()) 
             if 'nf' in key}

In [29]:
ae_train

{'callbacks': [<tensorflow.python.keras.callbacks.ReduceLROnPlateau at 0x7fc7f3772220>],
 'epochs': 60,
 'batch': 200}

In [30]:
nf_train

{'callbacks': [<tensorflow.python.keras.callbacks.ReduceLROnPlateau at 0x7fc7f37721c0>],
 'epochs': 50,
 'batch': 200}

In [32]:
ae_config['weight_reg']={'l1':1, 'l2':2}

In [33]:
test = ae_config

def interpret_config(config):
    for key in config:
        if 'activation' in key:
            config[key]= ACTIVATIONS[config[key]]
        if 'reg' in key:
            if len(config[key]) == 2 :
                config[key]= REGULARIZERS['l1_l2'](**config[key])
            elif len(config[key]) == 1:
                config[key]= REGULARIZERS[list(config[key])[0]](**config[key])



In [34]:
interpret_config(test)
test

{'input_dim': 47,
 'encoding_dim': 10,
 'units': [30, 20, 15],
 'hidden_activation': <function tensorflow.python.ops.gen_nn_ops.relu(features, name=None)>,
 'output_activation': <function tensorflow.python.ops.math_ops.sigmoid(x, name=None)>,
 'weight_reg': <tensorflow.python.keras.regularizers.L1L2 at 0x7fc7f33e39a0>,
 'bias_reg': {},
 'output_reg': {}}

In [37]:
test['weight_reg'].l2

array(2., dtype=float32)

In [ ]:
builder = PaeBuilder()

In [19]:
nf_train.keys()

dict_keys(['callbacks', 'epochs', 'batch'])

In [16]:
"NF:" in 'NF:n_dims'

True

input_dim
encoding_dim
units
hidden_activation
output_activation
weight_reg
bias_reg
output_reg
